In [1]:
import json
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
keypoint_names = ['Head', 'Tail']

# Dictionary to convert rectangles classes into keypoint classes because keypoint classes should start with 0
rectangles2keypoints = {1:0, 2:1}

def converter(file_labels, file_image, keypoint_names):

    img = cv2.imread(file_image)
    img_w, img_h = img.shape[1], img.shape[0]
    
    with open(file_labels) as f:
        lines_txt = f.readlines()
        lines = []
        for line in lines_txt:
            lines.append([int(line.split()[0])] + [round(float(el), 5) for el in line.split()[1:]])

    bboxes = []
    keypoints = []

    # In this loop we convert normalized coordinates to absolute coordinates
    for line in lines:
        # Number 0 is a class of rectangles related to bounding boxes.
        if line[0] == 0:
            x_c, y_c, w, h = round(line[1] * img_w), round(line[2] * img_h), round(line[3] * img_w), round(line[4] * img_h)
            bboxes.append([round(x_c - w/2), round(y_c - h/2), round(x_c + w/2), round(y_c + h/2)])

        # Other numbers are the classes of rectangles related to keypoints.
        # After convertion, numbers of keypoint classes should start with 0, so we apply rectangles2keypoints dictionary to achieve that.
        # In our case:
        # 1 is rectangle for head keypoint, which is 0, so we convert 1 to 0;
        # 2 is rectangle for tail keypoint, which is 1, so we convert 2 to 1.
        if line[0] != 0:
            kp_id, x_c, y_c = rectangles2keypoints[line[0]], round(line[1] * img_w), round(line[2] * img_h)
            keypoints.append([kp_id, x_c, y_c])

    # In this loop we are iterating over each keypoint and looking to which bounding box it matches.
    # Thus, we are matching keypoints and corresponding bounding boxes.
    keypoints_sorted = [[[] for _ in keypoint_names] for _ in bboxes]
    for kp in keypoints:
        kp_id, kp_x, kp_y = kp[0], kp[1], kp[2]
        for bbox_idx, bbox in enumerate(bboxes):
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
            if x1 < kp_x < x2 and y1 < kp_y < y2:
                keypoints_sorted[bbox_idx][kp_id] = [kp_x, kp_y, 1] # All keypoints are visible
                
    return bboxes, keypoints_sorted

In [3]:
def dump2json(bboxes, keypoints_sorted, file_json):
    annotations = {}
    annotations['bboxes'], annotations['keypoints'] = bboxes, keypoints_sorted
    with open(file_json, "w") as f:
        json.dump(annotations, f)

In [4]:
IMAGES = '../data/train/images'
LABELS = '../data/train/labels'
ANNOTATIONS = '../data/train/annotations'

files_names = [file.split('.jpg')[0] for file in os.listdir(IMAGES)]

for file in files_names:
    file_labels = os.path.join(LABELS, file + ".txt")
    file_image = os.path.join(IMAGES, file + ".jpg")
    print("Processing", file_labels, file_image)
    if os.path.exists(file_labels):
        print("Processing", file_labels, file_image)
        bboxes, keypoints_sorted = converter(file_labels, file_image, keypoint_names)
        dump2json(bboxes, keypoints_sorted, os.path.join(ANNOTATIONS, file + '.json'))
    else:
        print(f"Label file not found for {file_image}, skipping.")

Processing ../data/train/labels/image_3773_jpg.rf.8ebb5d29d7419ae805d37b9e3f62f624.txt ../data/train/images/image_3773_jpg.rf.8ebb5d29d7419ae805d37b9e3f62f624.jpg
Processing ../data/train/labels/image_3773_jpg.rf.8ebb5d29d7419ae805d37b9e3f62f624.txt ../data/train/images/image_3773_jpg.rf.8ebb5d29d7419ae805d37b9e3f62f624.jpg
Processing ../data/train/labels/image_1482_jpg.rf.87cd7eb14ffcfc1208e456e6bf7d6b42.txt ../data/train/images/image_1482_jpg.rf.87cd7eb14ffcfc1208e456e6bf7d6b42.jpg
Processing ../data/train/labels/image_1482_jpg.rf.87cd7eb14ffcfc1208e456e6bf7d6b42.txt ../data/train/images/image_1482_jpg.rf.87cd7eb14ffcfc1208e456e6bf7d6b42.jpg
Processing ../data/train/labels/image_2575_jpg.rf.55be035600ce3a3ba675659f65d9fe3a.txt ../data/train/images/image_2575_jpg.rf.55be035600ce3a3ba675659f65d9fe3a.jpg
Processing ../data/train/labels/image_2575_jpg.rf.55be035600ce3a3ba675659f65d9fe3a.txt ../data/train/images/image_2575_jpg.rf.55be035600ce3a3ba675659f65d9fe3a.jpg
Processing ../data/tra

In [5]:
IMAGES = '../data/test/images'
LABELS = '../data/test/labels'
ANNOTATIONS = '../data/test/annotations'

files_names = [file.split('.jpg')[0] for file in os.listdir(IMAGES)]

for file in files_names:
    file_labels = os.path.join(LABELS, file + ".txt")
    file_image = os.path.join(IMAGES, file + ".jpg")
    print("Processing", file_labels, file_image)
    if os.path.exists(file_labels):
        print("Processing", file_labels, file_image)
        bboxes, keypoints_sorted = converter(file_labels, file_image, keypoint_names)
        dump2json(bboxes, keypoints_sorted, os.path.join(ANNOTATIONS, file + '.json'))
    else:
        print(f"Label file not found for {file_image}, skipping.")

Processing ../data/test/labels/image_2953_jpg.rf.79dd93ed80660413ef52d9ad66a6e99d.txt ../data/test/images/image_2953_jpg.rf.79dd93ed80660413ef52d9ad66a6e99d.jpg
Processing ../data/test/labels/image_2953_jpg.rf.79dd93ed80660413ef52d9ad66a6e99d.txt ../data/test/images/image_2953_jpg.rf.79dd93ed80660413ef52d9ad66a6e99d.jpg
Processing ../data/test/labels/image_3820_jpg.rf.d4cd792f062c053f515940618eb6a073.txt ../data/test/images/image_3820_jpg.rf.d4cd792f062c053f515940618eb6a073.jpg
Processing ../data/test/labels/image_3820_jpg.rf.d4cd792f062c053f515940618eb6a073.txt ../data/test/images/image_3820_jpg.rf.d4cd792f062c053f515940618eb6a073.jpg
Processing ../data/test/labels/image_3791_jpg.rf.15a5f89d089c59e5564bd1f218f55cee.txt ../data/test/images/image_3791_jpg.rf.15a5f89d089c59e5564bd1f218f55cee.jpg
Processing ../data/test/labels/image_3791_jpg.rf.15a5f89d089c59e5564bd1f218f55cee.txt ../data/test/images/image_3791_jpg.rf.15a5f89d089c59e5564bd1f218f55cee.jpg
Processing ../data/test/labels/ima

In [6]:
IMAGES = '../data/valid/images'
LABELS = '../data/valid/labels'
ANNOTATIONS = '../data/valid/annotations'

files_names = [file.split('.jpg')[0] for file in os.listdir(IMAGES)]

for file in files_names:
    file_labels = os.path.join(LABELS, file + ".txt")
    file_image = os.path.join(IMAGES, file + ".jpg")
    print("Processing", file_labels, file_image)
    if os.path.exists(file_labels):
        print("Processing", file_labels, file_image)
        bboxes, keypoints_sorted = converter(file_labels, file_image, keypoint_names)
        dump2json(bboxes, keypoints_sorted, os.path.join(ANNOTATIONS, file + '.json'))
    else:
        print(f"Label file not found for {file_image}, skipping.")

Processing ../data/valid/labels/image_3218_jpg.rf.55f294057174508ba40fa4f44d0395ee.txt ../data/valid/images/image_3218_jpg.rf.55f294057174508ba40fa4f44d0395ee.jpg
Processing ../data/valid/labels/image_3218_jpg.rf.55f294057174508ba40fa4f44d0395ee.txt ../data/valid/images/image_3218_jpg.rf.55f294057174508ba40fa4f44d0395ee.jpg
Processing ../data/valid/labels/image_759_jpg.rf.4a2ceccb0c2bad6582a79c1cb6e7c8cf.txt ../data/valid/images/image_759_jpg.rf.4a2ceccb0c2bad6582a79c1cb6e7c8cf.jpg
Processing ../data/valid/labels/image_759_jpg.rf.4a2ceccb0c2bad6582a79c1cb6e7c8cf.txt ../data/valid/images/image_759_jpg.rf.4a2ceccb0c2bad6582a79c1cb6e7c8cf.jpg
Processing ../data/valid/labels/image_602_jpg.rf.ad12e7b0c4c167ba3a5e2e700ff15f0f.txt ../data/valid/images/image_602_jpg.rf.ad12e7b0c4c167ba3a5e2e700ff15f0f.jpg
Processing ../data/valid/labels/image_602_jpg.rf.ad12e7b0c4c167ba3a5e2e700ff15f0f.txt ../data/valid/images/image_602_jpg.rf.ad12e7b0c4c167ba3a5e2e700ff15f0f.jpg
